mmdetection으로 데이터학습을 통해서 모델을 만들어 봅시다.

https://mmdetection.readthedocs.io/en/latest/2_new_data_model.html



아래의 balloon dataset으로 instance segmentation을 학습해봅시다

https://github.com/matterport/Mask_RCNN/releases

데이터셋을 구글 드라이브에 압축을 풀어줍니다


위의 데이터셋을 COCO dataset으로 변환하여 진행합니다

json 파일은 vs code 등으로 Config 파일을 직접 확인하고 수정 가능합니다. 

Colab에서는 아래처럼 불러와서 확인할 수 있습니다

In [1]:
# 라벨링 json 파일 불러오기

import json

path = '/content/drive/MyDrive/DL 202201/balloon/'

with open(path +"val/via_region_data.json", "r") as f:
    anno = json.load(f)

print(anno)

{'24631331976_defa3bb61f_k.jpg668058': {'fileref': '', 'size': 668058, 'filename': '24631331976_defa3bb61f_k.jpg', 'base64_img_data': '', 'file_attributes': {}, 'regions': {'0': {'shape_attributes': {'name': 'polygon', 'all_points_x': [916, 913, 905, 889, 868, 836, 809, 792, 789, 784, 777, 769, 767, 777, 786, 791, 769, 739, 714, 678, 645, 615, 595, 583, 580, 584, 595, 614, 645, 676, 716, 769, 815, 849, 875, 900, 916, 916], 'all_points_y': [515, 583, 616, 656, 696, 737, 753, 767, 777, 785, 785, 778, 768, 766, 760, 755, 755, 743, 728, 702, 670, 629, 588, 539, 500, 458, 425, 394, 360, 342, 329, 331, 347, 371, 398, 442, 504, 515]}, 'region_attributes': {}}}}, '16335852991_f55de7958d_k.jpg1767935': {'fileref': '', 'size': 1767935, 'filename': '16335852991_f55de7958d_k.jpg', 'base64_img_data': '', 'file_attributes': {}, 'regions': {'0': {'shape_attributes': {'name': 'polygon', 'all_points_x': [588, 617, 649, 673, 692, 708, 722, 730, 737, 718, 706, 699, 697, 676, 650, 613, 580, 552, 534, 520,


COCO dataset 형식 참고

https://cocodataset.org/#format-data

```
{
    "images": [image],
    "annotations": [annotation],
    "categories": [category]
}


image = {
    "id": int,
    "width": int,
    "height": int,
    "file_name": str,
}

annotation = {
    "id": int,
    "image_id": int,
    "category_id": int,
    "segmentation": RLE or [polygon],
    "area": float,
    "bbox": [x,y,width,height],
    "iscrowd": 0 or 1,
}

categories = [{
    "id": int,
    "name": str,
    "supercategory": str,
}]
```

COCO dataset 예시

https://www.immersivelimit.com/tutorials/create-coco-annotations-from-scratch


In [3]:
# 한 이미지에 여러개의 라벨이 있는 경우를 고려하여 라벨을 만들어 봅시다. 
from PIL import Image

images = []
annos_coco = []
coco_format_json={}

image_id = 0
anno_id = 0
for key, value in anno.items():  
  
  print('image:', image_id)  # 진행현황 프린트
  
  # 이미지 정보 추출
  img = Image.open(path+'val/'+value['filename'])    
  image = {'id':image_id, 'width':img.width, 'height':img.height, 'file_name':value['filename'] }  # file_name 밑줄이 있음, 키 값을 정확히 일치시켜야 함
  images.append(image)
  print(image)
  
  # 라벨 정보 추출  


  annotations = value['regions'].items()
  for i, shape in annotations:
    
    print('anno',anno_id)    
    points_x = shape['shape_attributes']['all_points_x']
    points_y = shape['shape_attributes']['all_points_y']
  
    # 형식을 (x1, y1, x2, y2, x3, y3...) 형태로 변환합니다. 
    poly = [(x, y) for x, y in zip(points_x, points_y)]  # (x,y)의 리스트 만들어 줍니다. 
    poly = [p for x in poly for p in x]      

    # 박스 및 면적을 추출하기 위해 최대 최소를 추출
    x_min, y_min, x_max, y_max = min(points_x), min(points_y), max(points_x), max(points_y)

    # annotation dict 저장
    anno_coco = dict(   # dict()를 사용해서 딕셔너리를 만들 수 있으면 이때는 키=값 형태로 넣어준다. 키에 "" 없이 사용 가능
                  image_id=image_id,
                  id=anno_id,  # 데이터에 저장된 값으로 활용
                  category_id=0,  #라벨이 balloon 한개여서 고정값
                  bbox=[x_min, y_min, x_max - x_min, y_max - y_min], # 박스 저장 [x, y, width, height]
                  area=(x_max - x_min) * (y_max - y_min), #면적
                  segmentation=[poly], # 폴리곤
                  iscrowd=0) 
    annos_coco.append(anno_coco)                
    #print(annos_coco)            
    anno_id += 1
  image_id+=1

# 전체 내용을 json으로 저장
out_file = path + 'val/val.json'

coco_format_json = dict(
        images=images,
        annotations=annos_coco,
        categories=[{'id':0, 'name': 'balloon'}])

print(coco_format_json)

with open(out_file, "w") as f:  #json 저장
    json.dump(coco_format_json, f)


image: 0
{'id': 0, 'width': 1323, 'height': 2048, 'file_name': '24631331976_defa3bb61f_k.jpg'}
anno 0
image: 1
{'id': 1, 'width': 1536, 'height': 2048, 'file_name': '16335852991_f55de7958d_k.jpg'}
anno 1
anno 2
anno 3
anno 4
anno 5
anno 6
anno 7
image: 2
{'id': 2, 'width': 2048, 'height': 1365, 'file_name': '14898532020_ba6199dd22_k.jpg'}
anno 8
anno 9
anno 10
anno 11
anno 12
anno 13
anno 14
anno 15
anno 16
image: 3
{'id': 3, 'width': 2048, 'height': 1595, 'file_name': '8053085540_a72bd21a64_k.jpg'}
anno 17
anno 18
anno 19
anno 20
anno 21
anno 22
image: 4
{'id': 4, 'width': 2048, 'height': 1536, 'file_name': '6810773040_3d81036d05_k.jpg'}
anno 23
anno 24
anno 25
anno 26
image: 5
{'id': 5, 'width': 768, 'height': 1024, 'file_name': '5603212091_2dfe16ea72_b.jpg'}
anno 27
anno 28
anno 29
anno 30
anno 31
anno 32
anno 33
image: 6
{'id': 6, 'width': 683, 'height': 1024, 'file_name': '5555705118_3390d70abe_b.jpg'}
anno 34
image: 7
{'id': 7, 'width': 1024, 'height': 675, 'file_name': '48380316

In [ ]:
coco_format_json

{'annotations': [{'area': 246697,
   'bbox': [994, 619, 451, 547],
   'category_id': 0,
   'id': 0,
   'image_id': 0,
   'iscrowd': 0,
   'segmentation': [[1020,
     963,
     1000,
     899,
     994,
     841,
     1003,
     787,
     1023,
     738,
     1050,
     700,
     1089,
     663,
     1134,
     638,
     1190,
     621,
     1265,
     619,
     1321,
     643,
     1361,
     672,
     1403,
     720,
     1428,
     765,
     1442,
     800,
     1445,
     860,
     1441,
     896,
     1427,
     942,
     1400,
     990,
     1361,
     1035,
     1316,
     1079,
     1269,
     1112,
     1228,
     1129,
     1198,
     1134,
     1207,
     1144,
     1210,
     1153,
     1190,
     1166,
     1177,
     1166,
     1172,
     1150,
     1174,
     1136,
     1170,
     1129,
     1153,
     1122,
     1127,
     1112,
     1104,
     1084,
     1061,
     1037,
     1032,
     989,
     1020,
     963]]},
  {'area': 574600,
   'bbox': [135, 115, 676, 850],
  

In [4]:
coco_format_json['categories']

[{'id': 0, 'name': 'balloon'}]